# Example for life cycle assessment module of AeroMAPS


## Load and process

First, the user has to load the framework and generate a process.

In [1]:
%matplotlib widget
import pandas as pd
from aeromaps.core.process import create_process
from aeromaps.core.models import (
    models_traffic,
    models_efficiency_top_down,
    models_energy_without_fuel_effect,
    models_offset,
    models_climate_simple_gwpstar,
    models_sustainability,
)
from aeromaps.models.impacts.life_cycle_assessment.life_cycle_assessment import LifeCycleAssessment

In [2]:
extended_models = {
    "models_traffic": models_traffic,
    "models_efficiency_top_down": models_efficiency_top_down,
    "models_energy_without_fuel_effect": models_energy_without_fuel_effect,
    "models_offset": models_offset,
    "models_climate_simple_gwpstar": models_climate_simple_gwpstar,
    "models_sustainability": models_sustainability,
    "life_cycle_assessment": LifeCycleAssessment(
        "life_cycle_assessment"
    ),
}

Initial setup of EcoInvent/Premise already done, skipping. To reset the project use option generate(reset=True).


In [3]:
process = create_process(configuration_file="data_files/config.json", models=extended_models)

In [4]:
#process.list_float_inputs()

## Compute

Once all the parameters have been set up, the user can compute.

In [5]:
process.compute()

## Results

The user can then display the results. The user has access to float outputs but also to annual data outputs, with the possibility of choosing the output.

In [6]:
process.data['vector_outputs']

,annual_growth_rate_freight,annual_growth_rate_passenger,annual_growth_rate_passenger_long_range,annual_growth_rate_passenger_medium_range,annual_growth_rate_passenger_short_range,ask,ask_dropin_fuel,ask_electric,ask_hydrogen,ask_long_range,ask_long_range_dropin_fuel,ask_long_range_dropin_fuel_share,ask_long_range_electric,ask_long_range_electric_share,ask_long_range_hydrogen,ask_long_range_hydrogen_share,ask_medium_range,ask_medium_range_dropin_fuel,ask_medium_range_dropin_fuel_share,ask_medium_range_electric,ask_medium_range_electric_share,ask_medium_range_hydrogen,ask_medium_range_hydrogen_share,ask_short_range,ask_short_range_dropin_fuel,ask_short_range_dropin_fuel_share,ask_short_range_electric,ask_short_range_electric_share,ask_short_range_hydrogen,ask_short_range_hydrogen_share,biofuel_atj_efficiency,biofuel_atj_emission_factor,biofuel_atj_share,biofuel_ft_efficiency,biofuel_ft_msw_emission_factor,biofuel_ft_msw_share,biofuel_ft_others_emission_factor,biofuel_ft_others_share,biofuel_hefa_fog_emission_factor,biofuel_hefa_fog_share,biofuel_hefa_fuel_efficiency,biofuel_hefa_oil_efficiency,biofuel_hefa_others_emission_factor,biofuel_hefa_others_share,biofuel_mean_emission_factor,biofuel_share,biomass_atj_consumption,biomass_consumption,biomass_ft_consumption,biomass_hefa_fog_consumption,biomass_hefa_others_consumption,carbon_offset,carbon_offset_baseline_level_vs_2019,co2_emissions_2019technology,co2_emissions_2019technology_baseline3,co2_emissions_freight,co2_emissions_including_aircraft_efficiency,co2_emissions_including_energy,co2_emissions_including_load_factor,co2_emissions_including_operations,co2_emissions_long_range,co2_emissions_medium_range,co2_emissions_passenger,co2_emissions_per_rpk,co2_emissions_per_rtk,co2_emissions_short_range,co2_per_energy_mean,cumulative_carbon_offset,cumulative_co2_emissions,cumulative_co2_emissions_2019technology,cumulative_co2_emissions_2019technology_baseline3,cumulative_co2_emissions_including_energy,cumulative_co2_emissions_including_load_factor,dropin_fuel_consumption_liter_per_pax_100km,electricity_consumption,...,energy_consumption_short_range_without_operations,energy_consumption_without_operations,energy_per_ask_long_range_dropin_fuel,energy_per_ask_long_range_dropin_fuel_gain,energy_per_ask_long_range_electric,energy_per_ask_long_range_hydrogen,energy_per_ask_mean,energy_per_ask_mean_without_operations,energy_per_ask_medium_range_dropin_fuel,energy_per_ask_medium_range_dropin_fuel_gain,energy_per_ask_medium_range_electric,energy_per_ask_medium_range_hydrogen,energy_per_ask_short_range_dropin_fuel,energy_per_ask_short_range_dropin_fuel_gain,energy_per_ask_short_range_electric,energy_per_ask_short_range_hydrogen,energy_per_ask_without_operations_long_range_dropin_fuel,energy_per_ask_without_operations_long_range_electric,energy_per_ask_without_operations_long_range_hydrogen,energy_per_ask_without_operations_medium_range_dropin_fuel,energy_per_ask_without_operations_medium_range_electric,energy_per_ask_without_operations_medium_range_hydrogen,energy_per_ask_without_operations_short_range_dropin_fuel,energy_per_ask_without_operations_short_range_electric,energy_per_ask_without_operations_short_range_hydrogen,energy_per_rtk_freight_dropin_fuel,energy_per_rtk_freight_electric,energy_per_rtk_freight_hydrogen,energy_per_rtk_mean,energy_per_rtk_mean_without_operations,energy_per_rtk_without_operations_freight_dropin_fuel,energy_per_rtk_without_operations_freight_electric,energy_per_rtk_without_operations_freight_hydrogen,fuel_effect_correction_contrails,hydrogen_coal_ccs_share,hydrogen_coal_share,hydrogen_electrolysis_emission_factor,hydrogen_electrolysis_share,hydrogen_gas_ccs_share,hydrogen_gas_share,hydrogen_mean_emission_factor,kerosene_emission_factor,kerosene_share,level_carbon_offset,liquefaction_efficiency,load_factor,operations_contrails_gain,operations_contrails_overconsumption,operations_gain,reference_annual_growth_rate_aviation,relative_energy_per_ask_electric_wrt_dropin_l

In [7]:
process.climate_outputs_df

,aerosol_erf,annual_co2_erf,co2_emissions,co2_emissions_smooth,co2_erf,co2_erf_smooth,co2_h2o_erf,co2_h2o_nox_contrails_erf,co2_h2o_nox_erf,co2_total_erf_ratio,contrails_erf,cumulative_co2_emissions_smooth,cumulative_non_co2_equivalent_emissions,cumulative_total_equivalent_emissions,h2o_emissions,h2o_erf,historical_cumulative_co2_emissions_smooth,historical_cumulative_non_co2_equivalent_emissions,non_co2_equivalent_emissions,non_co2_erf_smooth,non_co2_erf_smooth_variation,nox_ch4_decrease_erf,nox_emissions,nox_erf,nox_long_term_o3_decrease_erf,nox_short_term_o3_increase_erf,nox_stratospheric_water_vapor_decrease_erf,soot_emissions,soot_erf,sulfur_emissions,sulfur_erf,temperature_increase_from_aviation,temperature_increase_from_co2_from_aviation,temperature_increase_from_non_co2_from_aviation,total_aircraft_distance,total_co2_equivalent_emissions_ratio,total_equivalent_emissions,total_erf,total_erf_smooth
1940,-0.187788,0.0323052,35.1359,38.0251,0.0323052,0.0681302,0.0899335,1.37636,0.58773,2.718,0.788625,NaN,NaN,NaN,11.0824,0.0576283,0.0380251,0.0680493,68.0493,1.25134,0.062567,-0.730102,0.136301,0.497797,-0.360903,1.69666,-0.107856,0.000270082,0.0271973,0.0108033,-0.214985,4.77334e-05,1.71113e-05,3.06222e-05,7.45392e+08,3.01897,106.074,1.18857,1.31947
1941,-0.202844,0.0348952,37.9528,39.5861,0.0672004,0.0874945,0.129449,1.51901,0.667155,5.10578,0.851851,NaN,NaN,NaN,11.9709,0.0622485,0.0776112,0.138892,70.8429,1.30271,0.0651356,-0.788636,0.147229,0.537706,-0.389837,1.83268,-0.116503,0.000291735,0.0293778,0.0116694,-0.232221,9.74265e-05,3.4925e-05,6.25015e-05,8.05153e+08,2.90964,110.429,1.31616,1.39021
1942,-0.219057,0.0376844,40.9865,41.2313,0.104885,0.107905,0.172109,1.67274,0.752795,7.21513,0.919941,NaN,NaN,NaN,12.9277,0.0672242,0.118842,0.212679,73.7871,1.35685,0.0678426,-0.851673,0.158997,0.580686,-0.420997,1.97917,-0.125815,0.000315054,0.031726,0.0126022,-0.250783,0.000149185,5.34791e-05,9.57057e-05,8.6951e+08,2.80626,115.018,1.45368,1.46476
1943,-0.236603,0.0407028,44.2693,44.5315,0.145588,0.148849,0.218196,1.83902,0.845392,9.08552,0.993624,NaN,NaN,NaN,13.9632,0.0726085,0.163374,0.292372,79.6931,1.46545,0.0732727,-0.919888,0.171732,0.627196,-0.454717,2.13769,-0.135892,0.000340289,0.0342671,0.0136115,-0.27087,0.000205086,7.35183e-05,0.000131568,9.39153e+08,2.80611,124.225,1.60241,1.6143
1944,-0.255538,0.0439602,47.8121,48.0938,0.189548,0.193068,0.267967,2.0185,0.945357,10.7517,1.07314,NaN,NaN,NaN,15.0806,0.0784193,0.211468,0.378441,86.0682,1.58268,0.0791342,-0.993506,0.185476,0.67739,-0.491108,2.30877,-0.146768,0.000367522,0.0370094,0.0147009,-0.292547,0.000265459,9.51605e-05,0.000170298,1.01431e+09,2.80603,134.162,1.76296,1.77575
1945,-0.275978,0.0474766,51.6366,51.9422,0.237024,0.240826,0.321717,2.21228,1.05329,12.2411,1.15898,NaN,NaN,NaN,16.287,0.0846921,0.26341,0.471396,92.9552,1.70933,0.0854664,-1.07298,0.200312,0.731575,-0.530392,2.49345,-0.158508,0.00039692,0.0399698,0.0158768,-0.315948,0.000330663,0.000118534,0.000212128,1.09545e+09,2.8061,144.897,1.9363,1.95015
1946,-0.298041,0.051272,55.7645,56.0961,0.288296,0.292403,0.379759,2.42145,1.16982,13.577,1.25164,NaN,NaN,NaN,17.5889,0.0914625,0.319506,0.571785,100.389,1.84603,0.0923013,-1.15875,0.216326,0.790058,-0.572792,2.69278,-0.171179,0.00042865,0.0431651,0.017146,-0.341206,0.000401081,0.000143778,0.000257303,1.18302e+09,2.80617,156.485,2.12341,2.13843
1947,-0.321898,0.0553761,60.2283,60.5837,0.343673,0.348105,0.442456,2.64758,1.29576,14.7773,1.35182,NaN,NaN,NaN,18.9969,0.0987838,0.38009,0.680205,108.42,1.99371,0.0996853,-1.25151,0.233642,0.8533,-0.618642,2.90833,-0.184882,0.000462962,0.0466203,0.0185185,-0.368518,0.000477132,0.00017104,0.000306092,1.27772e+09,2.80605,169.004,2.32568,2.34181
1948,-0.347608,0.059799,65.0388,65.4288,0.403472,0.408263,0.510145,2.89139,1.4316,15.8611,1.4598,NaN,NaN,NaN,20.5142,0.106674,0.445518,0.797296,117.091,2.15315,0.107658,-1.35146,0.252303,0.921453,-0.668054,3.14062,-0.199648,0.000499939,0.0503439,0.0199976,-0.397952,0.000559266

In [8]:
process.lca_outputs_df

2020  \
method                                                                                                 phase                             
acidification - accumulated exceedance (AE)[mol H+-Eq]                                                 _other_                       0   
                                                                                                       aircraft_production      23,560   
                                                                                                       airport             5.50102e+07   
                                                                                                       combustion            0.0115902   
                                                                                                       kerosene_production  0.00379697   
                                                                                                       *sum*               5.50337e+07   
climate change - global warming potential (GWP100)[kg CO2-Eq]                                          _other_                       0   
                                                                                                       aircraft_production 3.65087e+06   
                                                                                                       airport             1.47062e+10   
                                                                                                       combustion              3.18593   
                                                                                                       kerosene_production    0.850442   
                                                                                                       *sum*               1.47099e+10   
water use - user deprivation potential (deprivation-weighted water consumption)[m3 world eq. deprived] _other_                       0   
                                                                                                       aircraft_production     782,115   
                                                                                                       airport             4.65677e+09   
                                                                                                       combustion            0.0528285   
                                                                                                       kerosene_production   0.0627486   
                                                                                                       *sum*               4.65755e+09   
land use - soil quality index[dimensionless]                                                           _other_                       0   
                                                                                                       aircraft_production  9.0326e+06   
                                                                                                       airport             1.86524e+11   
                                                                                                       combustion                    0   
                                                                                                       kerosene_production     2.54932   
                                                                                                       *sum*               1.86533e+11   

                                                                                                                                  2021  \
method                                                                                                 phase                             
acidification - accumulated exceedance (AE)[mol H+-Eq]                                                 _other_             0.000112874   
                                                                                                       aircraft_production    2

# Draft

In [ ]:
from functools import wraps

def kwargs_decorator():
    def wrapper(f):
        @wraps(f)
        def inner_wrapper(*args, **kwargs):
            dict_kwargs = args[0].lca_params
            new_kwargs = {**dict_kwargs, **kwargs}
            return f(*args, **new_kwargs)
        return inner_wrapper
    return wrapper

class Test:
    def __init__(self):
        self.lca_params = {'a': 1.0}
        
    @kwargs_decorator()
    def foo(self, **kwargs):
        print(kwargs)

In [ ]:
test = Test()
test.foo()

In [ ]:
from functools import wraps
import lca_algebraic as agb

def kwargs_decorator(dict_kwargs):
    def wrapper(f):
        @wraps(f)
        def inner_wrapper(*args, **kwargs):
            # dict_kwargs = args[0].my_dict  # args[0] refer to 'self'
            new_kwargs = {**dict_kwargs, **kwargs}
            return f(*args, **new_kwargs)
        return inner_wrapper
    return wrapper


def args_decorator(list_args):
    def wrapper(f):
        @wraps(f)
        def inner_wrapper(*args, **kwargs):
            # dict_kwargs = args[0].my_dict  # args[0] refer to 'self'
            new_args = {*args, *list_args}
            return f(*new_args, **kwargs)
        return inner_wrapper
    return wrapper


class Test:
        
    @args_decorator(agb.all_params().keys())
    def foo(self, *args, **kwargs):
        return args
    
    
test = Test()
test.__init__()
res = test.foo()
res

In [ ]:
from inspect import getfullargspec

getfullargspec(test.foo)

In [ ]:
list(agb.all_params().keys())

In [ ]:
class Test:
    def init(self):
        self.my_list = ['a', 'b']
    
    def foo(self, *args, **kwargs):
        
        
        def foo2(
        
        return args
    

In [ ]:
test = ['q', 1.0]

any(isinstance(elem, str) for elem in test)